# Sprint アンサンブル学習
3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。

- ブレンディング
- バギング
- スタッキング

### 小さなデータセットの用意
以前も利用した回帰のデータセットを用意します。<br>
<br>
House Prices: Advanced Regression Techniquesのtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。<br>
<br>
train.csvを学習用（train）8割、検証用（val）2割に分割してください。<br>
<br>
単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
house_data = pd.read_csv('/Users/tamiyagt/Documents/machine learning/02_Kaggle/house prices/train.csv')
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

#GrLivArea、YearBuilt、SalePriceを抽出
train = house_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

train.head()

,GrLivArea,YearBuilt,SalePrice
0,1710,2003,208500
1,1262,1976,181500
2,1786,2001,223500
3,1717,1915,140000
4,2198,2000,250000


In [21]:
from sklearn.model_selection import train_test_split

# DataFrameをndarrayに変換
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])

# データの分割（今回は8：2の割合）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)

(1168, 2)
(1168,)


In [36]:
# 交差検証用関数を作成
def cross_validate(model, X, y, split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    
    for train_id, val_id in kf.split(X, y):
        train_X = X[train_id]
        train_y = y[train_id]
        val_X = X[val_id]
        val_y = y[val_id]

        # 学習、推定
        model.fit(train_X, train_y)
        pred = model.predict(val_X)
        
        results.append(mean_squared_error(val_y, pred))
    return np.array(results)

## 【問題1】ブレンディングのスクラッチ実装
**ブレンディング**をスクラッチ実装し、単一モデルより精度があがる例を**最低3つ**示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

### ブレンディングとは
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。<br>

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。<br>
<br>
回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。<br>
<br>
**《補足》**<br>
<br>
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。<br>
[sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from numpy import random

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [24]:
# Blendingクラスの実装

class Blending():
    def __init__(self, model_list=None):
        self.model_list = model_list
    
    def fit(self, X, y):
        for model in self.model_list:
            model.fit(X, y)
            
    def predict(self, X, metric='mean', weight=None):
        
        preds = np.empty((len(X), len(self.model_list)))
        
        for i, model in enumerate(self.model_list):
            preds[:, i] = model.predict(X)
        
        
        if metric == 'mean':
            return preds.mean(axis=1)
        elif metric == 'median':
            return np.median(preds, axis=1)
        elif metric == 'max':
            return preds.max(axis=1)
        elif metric == 'min':
            return preds.min(axis=1)
        elif metric == 'weighted':
            return np.sum(preds*weight, axis=1) / np.sum(weight)

>《例１》　線形回帰、SVR、決定木をブレンド

In [7]:
# 各モデルをインスタンス化
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor()

models = [linreg, svr, tree]
labels = ['Linear Regression', 'SVR', 'Decision Tree']

# 各モデルのブレンドモデルをインスタンス化
blend = Blending(models)

# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print("{} MSE: {:.2e}".format(label, mean_squared_error(y_test, pred)))
print('\n')

# ブレンドモデルの学習・推定
metrics = ['mean', 'median', 'max', 'min']

for metric in metrics:
    blend.fit(X_train, y_train)
    pred = blend.predict(X_test, metric=metric)
    print("Blended {} model MSE: {:.2e}".format(metric, mean_squared_error(y_test, pred)))

Linear Regression MSE: 1.72e+09
SVR MSE: 5.85e+09
Decision Tree MSE: 2.07e+09


Blended mean model MSE: 1.99e+09
Blended median model MSE: 1.78e+09
Blended max model MSE: 2.94e+09
Blended min model MSE: 5.00e+09


>デフォルトパラメータのモデル３種を４種のブレンド法で推定。中央値を求めたブレンドが最もMSEが低い

>《例２》　標準化し、モデルをブレンド

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# 各モデルをインスタンス化
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor()

models = [linreg, svr, tree]
labels = ['Linear Regression', 'SVR', 'Decision Tree']

# 各モデルのブレンドモデルをインスタンス化
blend = Blending(models)

In [13]:
# 特徴量を標準化
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# 目的変数を対数変換
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    print("{} MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))

print('\n')

# ブレンドモデルの学習・推定
metrics = ['mean', 'median', 'max', 'min']

for metric in metrics:
    blend.fit(X_train_std, y_train_log)
    pred = blend.predict(X_test_std, metric=metric)
    print("Blended {} model MSE: {:.4f}".format(metric, mean_squared_error(y_test_log, pred)))

Linear Regression MSE: 0.0455
SVR MSE: 0.0465
Decision Tree MSE: 0.0808


Blended mean model MSE: 0.0484
Blended median model MSE: 0.0458
Blended max model MSE: 0.0558
Blended min model MSE: 0.0724


>標準化・対数変換後はSVR単独の方がブレンドモデルよりスコアが良い。ブレンディング法は例１同様中央値が最も良い結果となった。

>《例３》重みを加えたブレンド

In [23]:
# 各モデルをインスタンス化
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor(max_depth=5)

models = [linreg, svr, tree]
labels = ['Linear Regression', 'SVR', 'Decision Tree']
weights = [0.4, 0.4, 0.2]

# 各モデルのブレンドモデルをインスタンス化
blend = Blending(models)

# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    print("{} MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))

print('\n')

# ブレンドモデルの学習・推定
blend.fit(X_train_std, y_train_log)

metrics = ['mean', 'median', 'max', 'min', 'weighted']

for metric in metrics:
    blend.fit(X_train_std, y_train_log)
    pred = blend.predict(X_test_std, metric=metric, weight=weights)
    print("Blended {} model MSE: {:.4f}".format(metric, mean_squared_error(y_test_log, pred)))

Linear Regression MSE: 0.0496
SVR MSE: 0.0470
Decision Tree MSE: 0.0527


Blended mean model MSE: 0.0472
Blended median model MSE: 0.0468
Blended max model MSE: 0.0510
Blended min model MSE: 0.0515
Blended weighted model MSE: 0.0470


>《例４》SVMのカーネルをブレンド

In [126]:
# 各モデルをインスタンス化
svr_rbf = SVR(kernel='rbf')
svr_lin = SVR(kernel='linear')
svr_poly = SVR(kernel='poly')
svr_sig = SVR(kernel='sigmoid')


models = [svr_rbf, svr_lin, svr_poly, svr_sig]
labels = ['rbf', 'linear', 'polynomial', 'sigmoid']
# weights = [0.3, 0.1, 0.2, 0.3, 0.1]

# 各モデルのブレンドモデルをインスタンス化
blend = Blending(models)

# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    print("{} MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))

print('\n')

# ブレンドモデルの学習・推定
blend.fit(X_train_std, y_train_log)

metrics = ['mean', 'median']

for metric in metrics:
    blend.fit(X_train_std, y_train_log)
    pred = blend.predict(X_test_std, metric=metric, weight=weights)
    print("Blended {} model MSE: {:.4f}".format(metric, mean_squared_error(y_test_log, pred)))

rbf MSE: 0.0470
linear MSE: 0.0496
polynomial MSE: 0.0819
sigmoid MSE: 895.9715


Blended mean model MSE: 56.5664
Blended median model MSE: 0.0577


## 【問題2】バギングのスクラッチ実装
**バギング**をスクラッチ実装し、単一モデルより精度があがる例を**最低1つ**示してください。

### バギングとは
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブレンディングと同様の実装になります。

In [38]:
# Baggingクラスの実装

import copy

class Bagger():
    def __init__(self, model, n_samples, n_bootstraps=10):
        self.model = model
        self.n_bootstraps = n_bootstraps
        self.n_samples = n_samples
        self.model_list = [copy.deepcopy(model) for _ in range(n_bootstraps)]  # Shallow Copyだと変数のみCopyされるため、Deepcopyでデータ毎複製
    

    def fit(self, X, y):
        
        for i in range(self.n_bootstraps):
            
            bag_id = random.randint(len(X), size=self.n_samples)
            
            self.model_list[i].fit(X[bag_id], y[bag_id])


    def predict(self, X, metric='mean'):
        
        preds = np.empty((len(X), len(self.model_list)))
        
        for i, model in enumerate(self.model_list):
            preds[:, i] = model.predict(X)
        
        if metric == 'mean':
            return preds.mean(axis=1)
        elif metric == 'median':
            return np.median(preds, axis=1)
        elif metric == 'max':
            return preds.max(axis=1)
        elif metric == 'min':
            return preds.min(axis=1)

In [150]:
# 各モデルをインスタンス化
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor()

models = [linreg, svr, tree]
labels = ['Linear Regression', 'SVR', 'Decision Tree']

results_standalone = pd.DataFrame(index=[labels], columns=['train score', 'test score'])
results_bagging = pd.DataFrame(index=[labels], columns=['train score', 'test score'])


# 各単一モデルの学習・推定
for model, label in zip(models, labels):
    
    # 5分割交差検証
    results_standalone['train score'][label] = cross_validate(model, X_train_std, y_train_log).mean()

    # テストデータ検証
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    results_standalone['test score'][label] = mean_squared_error(y_test_log, pred)
    
#     print("{} MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))
print('\n')


# Baggingモデルの学習・推定

for model, label in zip(models, labels):
        
    bagging = Bagger(model, n_samples=500, n_bootstraps=10)
    
    # 5分割交差検証
    results_bagging['train score'][label] = cross_validate(bagging, X_train_std, y_train_log).mean()

    bagging.fit(X_train_std, y_train_log)
    pred = bagging.predict(X_test_std)
    results_bagging['test score'][label] = mean_squared_error(y_test_log, pred)


#     print("Bagged {} Train MSE: {:.4f}".format(label, cross_validate(bagging, X_train_std, y_train_log).mean()))
#     print("Bagged {} Test MSE: {:.4f}".format(label, mean_squared_error(y_test_log, pred)))

print("単一モデルの交差検証＆推定結果")
print(results_standalone)
print('\n')
print("バギングモデルの交差検証＆推定結果")
print(results_bagging)



単一モデルの交差検証＆推定結果
                  train score test score
Linear Regression   0.0485171  0.0495937
SVR                 0.0416455  0.0470362
Decision Tree       0.0685942  0.0770671


バギングモデルの交差検証＆推定結果
                  train score test score
Linear Regression   0.0482951  0.0498268
SVR                 0.0423308  0.0477689
Decision Tree       0.0434625  0.0521715


>Baggingは決定木に最も影響を与えた。

## 【問題3】スタッキングのスクラッチ実装
**スタッキング**をスクラッチ実装し、単一モデルより精度があがる例を**最低1つ**示してください。<br>
<br>
### スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは$K_0=3, M_0=2$程度にします。<br>
<br>
**《学習時》**<br>
<br>
（ステージ 0）
- 学習データを$K_0$個に分割する。
- 分割した内の$(K_0-1)$個をまとめて学習用データ、残り1個を推定用データとする組み合わせが$K_0$個作れる。
- あるモデルのインスタンスを$K_0$個用意し、異なる学習用データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り1個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも$K_0$個用意し、同様のことを行う。モデルが$M_0$個あれば、$M_0$個のブレンドデータが得られる。

（ステージ $n$）
- ステージ$n-1$のブレンドデータを$M_{n-1}$次元の特徴量を持つ学習用データと考え、$K_n$個に分割する。以下同様である。

（ステージ $N$）＊最後のステージ
- ステージ$N-1$の$M_{N-1}$個のブレンドデータを$M_{N-1}$次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

<br>

**《推定時》**

（ステージ 0）
- テストデータを$K_0\times M_0$個の学習済みモデルに入力し、$K_0\times M_0$個の推定値を得る。これを$K_0$の軸で平均値を求め$M_0$次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$）
- ステージ$n-1$で得たブレンドテストを$K_n\times M_n$個の学習済みモデルに入力し、$K_n\times M_n$個の推定値を得る。これを$K_n$の軸で平均値を求め$M_0$次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$）＊最後のステージ
- ステージ$N-1$で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [50]:
from sklearn.model_selection import KFold

class Stacked():
    def __init__(self, base_models, final_model, k=3):
        self.base_models = base_models
        self.final_model = final_model
        self.k = k
        self.model_inst = [[copy.deepcopy(self.base_models[i]) for _ in range(k)] for i in range(len(self.base_models))]


    def fit(self, X, y):
        kfold = KFold(n_splits=self.k, shuffle=True)
        
        X_train_f = np.empty((len(X), len(self.base_models)))
        
        for i in range(len(self.model_inst)):
            for model, (train_id, test_id) in zip(self.model_inst[i], kfold.split(X)):
            
                X_train, y_train = X[train_id], y[train_id]
                X_test, y_test = X[test_id], y[test_id]

                model.fit(X_train, y_train)
                X_train_f[test_id, i] = model.predict(X_test)
        
        self.final_model.fit(X_train_f, y)
        
    
    def predict(self, X):
        X_test_f = np.empty((len(X), len(self.base_models)))
        
        for i in range(len(self.model_inst)):
            
            pred = np.empty((len(X), self.k))
            
            for j, model in enumerate(self.model_inst[i]):
                pred[:, j] = model.predict(X)
                
            X_test_f[:, i] = pred.mean(axis=1)
            
        return self.final_model.predict(X_test_f)        

In [76]:
linreg = LinearRegression()
svr = SVR()
tree = DecisionTreeRegressor(max_depth=10)

stacked = Stacked([tree, svr], linreg, k=10)

models = [linreg, svr, tree, stacked]
labels = ['Linear Regression', 'SVR', 'Decision Tree', 'Stacked']

results_stacked = pd.DataFrame(index=[labels], columns=['train score', 'test score'])

for model, label in zip(models, labels):
   # 5分割交差検証
    results_stacked['train score'][label] = cross_validate(model, X_train_std, y_train_log).mean()

    # テストデータ検証
    model.fit(X_train_std, y_train_log)
    pred = model.predict(X_test_std)
    results_stacked['test score'][label] = mean_squared_error(y_test_log, pred)
    
print("スタッキングモデルの交差検証＆推定結果")
print(results_stacked)

スタッキングモデルの交差検証＆推定結果
                  train score test score
Linear Regression   0.0493836  0.0455039
SVR                 0.0415591  0.0465108
Decision Tree       0.0609494  0.0664328
Stacked             0.0403909  0.0462007


>スタッキングモデルが単一モデルと比べ、訓練データのクロスバリデーションと検証データに対する推定共に良い結果となった。